In [1]:
from kdrag.all import *
import kdrag.theories.int as int_
n,m,k,p,q = smt.Ints("n q k p q")

even = kd.define("even", [n], smt.Exists([k], n == 2*k))
odd = kd.define("odd", [n], smt.Exists([k], n == 2*k + 1))

N,M,K = kd.FreshVars("N M K", smt.IntSort())

with kd.tactics.Goal(sig=[N],ctx=[odd(N-1)],goal=even(N)).proof() as l:
    l.unfold(odd, at=0)
    l.unfold(even)
    _k = l.obtain(-1)
    l.exists(1 + _k)
    l.auto()
    odd_dec_even = l.qed().forall([N])

with kd.tactics.Goal(sig=[N],ctx=[even(N-1)],goal=odd(N)).proof() as l:
    l.unfold(even, at=0)
    l.unfold(odd)
    _k = l.obtain(-1)
    l.exists(_k)
    l.auto()
    even_dec_odd = l.qed().forall([N])

with kd.tactics.Goal(sig=[N],ctx=[odd(N+1)],goal=even(N)).proof() as l:
    l.unfold(odd, at=0)
    l.unfold(even)
    print(l)
    _k = l.obtain(-1)
    l.exists(_k)
    l.auto()
    odd_inc_even = l.qed().forall([N])

with kd.tactics.Goal(sig=[N],ctx=[even(N+1)],goal=odd(N)).proof() as l:
    l.unfold(even, at=0)
    l.unfold(odd)
    _k = l.obtain(-1)
    l.exists(_k - 1)
    l.auto()
    even_inc_odd = l.qed().forall([N])



[N!38] ; [Exists(k, N!38 + 1 == 2*k + 1)] ?|= Exists(k, N!38 == 2*k)


In [2]:
with kd.tactics.Goal(
        sig=[N],
        ctx=[],
        goal=smt.Or(even(N), odd(N))).proof() as l:
    l.induct(N)
    _n = l.fix()
    #l.simp()
    #l.auto(by=[odd_even(_n+1), even_odd(_n+1)])
    #print(l)
    l.intros()
    l.simp()
    l.left()
    #l.apply(odd_even(_n)) # shouldn't fail. What's up?
    l.apply(odd_inc_even)
    l.simp(at=0)
    l.split(at=0)
    l.cases(odd(_n))
    l.auto()
    l.have(even(_n), by=[])
    l.apply(2)
    l.apply(even_inc_odd)
    l.auto()
    # case 0
    l.auto(unfold=1)
    # case n > 0
    _n = l.fix()
    l.intros()
    l.simp()
    l.left()
    l.apply(odd_dec_even)
    l.cases(odd(_n))
    l.auto()
    l.have(even(_n), by=[])
    l.apply(1)
    l.apply(even_dec_odd)
    l.auto()
    even_or_odd = l.qed().forall([N])

    # l.apply(-1) # todo make apply not work.
    #l.auto()
even_or_odd

|= ForAll(N!38, Or(even(N!38), odd(N!38)))

In [ ]:
from functools import cached_property

class AbstractGroup():
    G = smt.DeclareSort("AbstractGroup")
    A,B,C = kd.FreshVars("A B C", G)
    a,b,c = smt.Consts("a b c", G)
    mul = smt.Function("mul", G, G, G)
    kd.notation.mul.register(G, mul)
    mul_assoc = kd.axiom(smt.ForAll([a,b,c], (a * b) * c == a * (b * c)))
    
    mul_comm = kd.axiom(smt.ForAll([a,b], a * b == b * a))


    #with kd.tactics.Goal(fixes=[A,B,C],
    #    shows
    with kd.Lemma(A * (B * C) == B * (A * C)) as l:
        l.have((B * A) * C == (A * B) * C, by=[mul_comm(A,B)])
        l.auto(by=[mul_assoc(B,A,C), mul_assoc(A,B,C)])
        left_commute = l.qed().forall([A,B,C])

    e = smt.Const("e", G)
    inv = smt.Function("inv", G, G)
    group_left_neutral = kd.axiom(smt.ForAll([a], e * a == a))
    left_inverse = kd.axiom(smt.ForAll([a], inv(a) * a == e))

    with kd.Lemma((A * B == A * C) == (B == C)) as l:
        l.split()
        l.intros() #(A * B == A * C)
        l.have(inv(A) * (A * B) == inv(A) * (A * C), by=[])
        l.have((inv(A) * A) * B == (inv(A) * A) * C, by=[mul_assoc(inv(A), A, B), mul_assoc(inv(A), A, C)])
        l.show(B == C, by=[left_inverse, group_left_neutral])
        #l.show(B == C, by=[left_inverse(A), group_left_neutral(B), group_left_neutral(C)])
        l.auto()
        left_cancel = l.qed().forall([A,B,C])

    idem = kd.axiom(smt.ForAll([a], a * a == a))

    left_idem = kd.prove(A * (A * B) == A * B, by=[idem(A), mul_assoc(A,A,B)]).forall([A,B])
    right_idem = kd.prove((B * A) * A == B * A, by=[idem(A), mul_assoc(B,A,A)]).forall([A,B])

    le = kd.notation.le.define([a,b], a == a * b)
    lt = kd.notation.lt.define([a,b], smt.And(a != b, a == a * b))
    order_iff = le.defn
    strict_order_iff = lt.defn

    refl = kd.prove(A <= A, unfold=1, by=[idem(A)]).forall([A])
    
    strict_iff_order = kd.prove((A < B) == smt.And(A <= B, A != B), unfold=1)
   
    with kd.Lemma(kd.QImplies(A <= B, B <= A, A == B)) as l:
        l.intros()
        l.have(A == A * B, unfold=1)
        l.have(A * B == B, unfold=1, by=[mul_comm])
        l.show(A == B, by=[])
        antisymm = l.qed().forall([A,B])
    
    with kd.Lemma(kd.QImplies(A <= B, B <= C, A <= C)) as l:
        l.intros()
        l.have(A == A * B, unfold=1)
        l.have(B == B * C, unfold=1)
        l.have(A == A * (B * C), by=[])
        l.have(A == (A * B) * C, by=[mul_assoc])
        l.have(A == A * C, by=[])
        l.have(A <= C, unfold=1)
        trans = l.qed().forall([A,B,C])

    cobounded1 = kd.prove(A * B <= B, unfold=1, by=[mul_assoc, idem(B), mul_comm]).forall([A,B])
    cobounded2 = kd.prove(B * A <= B, unfold=1, by=[mul_comm(A,B), cobounded1(A,B)]).forall([A,B])





    



AbstractGroup.left_commute
AbstractGroup.left_cancel
AbstractGroup.trans


Goal [And(le(A!203, B!204), le(B!204, C!205)),
 A!203 == mul(A!203, B!204),
 B!204 == mul(B!204, C!205),
 A!203 == mul(A!203, mul(B!204, C!205)),
 A!203 == mul(mul(A!203, B!204), C!205),
 A!203 == mul(A!203, C!205),
 le(A!203, C!205)]
?|= le(A!203, C!205)


|= ForAll([A!203, B!204, C!205],
       Implies(And(le(A!203, B!204), le(B!204, C!205)),
               le(A!203, C!205)))